In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np; np.set_printoptions(precision=4); np.random.seed(0)
import torch; torch.set_printoptions(precision=4)
seed = 1

torch.manual_seed(seed)
import torch.nn as nn
import matplotlib.pyplot as plt; plt.rc('font', size=12)
import matplotlib 
from matplotlib.font_manager import FontProperties
from mpl_toolkits import mplot3d
import matplotlib.pylab as pl
import seaborn as sns
import time
import sys
import itertools
import random; random.seed(0)
import scipy
import os
from textwrap import wrap
from scipy.stats import wilcoxon


from functions import *

print(torch.__version__)
print(sys.version)
                
%matplotlib inline

torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True) 
torch.backends.cudnn.deterministic = True    

1.13.1+cu116
3.10.8 (main, Nov 24 2022, 14:13:03) [GCC 11.2.0]


# Supplementary Figure 7a: Joint distribution of selectivity indices for different task variables

In [ ]:
# rule selectivity distribution 
with open('/.../rule_selectivity_distribution.pickle', 'rb') as handle:
    all_data_sm_rule_resp_sel = pickle.load(handle)
cgs = model.rnn.cell_group_list

rule_sel_across_models = {}
for cg in cgs:
    rule_sel_across_models[cg] = []
for x in all_data_sm_rule_resp_sel: 
    for cg in cgs:
        rule_sels = [x['all_sels']['rule_normalized_activity'][n] for n in model.rnn.cg_idx[cg]]
        rule_sel_across_models[cg].extend(rule_sels)
    fig.tight_layout()
    plt.show()


for cg in cgs:
    fig, ax=plt.subplots(1, 1, figsize=[4, 2.5])
    fig.patch.set_facecolor('white')
    fig.suptitle('Across all models, {}'.format(cg))
    ax.set_title(cg)
    ax.hist(rule_sel_across_models[cg], color='k', bins=20)
    make_pretty_axes(ax)
    fig.tight_layout()
    plt.show()
    fig.savefig('/home/yl4317/Documents/two_module_rnn/figs/rule_sel_dist_{}.pdf'.format(cg))

# Figure 5b joint distribution of rule, response and shared feature selectivity

In [ ]:
# rule selectivity x response selectivity x feature selectivity across all models

rule_sel_all = []
resp_sel_all = []
feature_sel_all = []

cg_idx = model.rnn.cg_idx['sr_esoma'].tolist() + model.rnn.cg_idx['sr_pv'].tolist() + model.rnn.cg_idx['sr_sst'].tolist() + model.rnn.cg_idx['sr_vip'].tolist()

for x in all_data_sm_rule_resp_sel:
    rule_sel_all.extend(x['all_sels']['rule_normalized_activity'][n] for n in cg_idx)
    resp_sel_all.extend(x['all_sels']['resp_normalized'][n]['max'] for n in cg_idx)
    feature_sel_all.extend(x['all_sels']['common_feature'][n] for n in cg_idx)


fig = plt.figure()
ax = fig.add_subplot(projection='3d')
fig.patch.set_facecolor('white')
ax.scatter(xs=np.array(rule_sel_all), ys=np.array(resp_sel_all), zs=np.array(feature_sel_all), color='k')  
ax.set_xlabel('Rule selectivity')
ax.set_ylabel('Response selectivity')
ax.set_zlabel('Shared feature selectivity')
ax.set_xlim([-1, 1])
ax.set_ylim([0, 1])
ax.set_zlim([-1, 1])
make_pretty_axes(ax)
fig.tight_layout()
plt.show()

